In [1]:
import sys

# variables
charClass = 0
lexeme = []
nextChar = ' '
token = 0
nextToken = ' '
index = 0
chars = []
tokenString = ' '
idNum = 0
constNum = 0
opNum = 0
perLine = []
lineNum = 0
idArray = {}
constValue = 0
add_op = 0
mult_op = 0
grammar = 0
message = ' '
lastIndex = 0

# Character classes
LETTER = 0
DIGIT = 1
UNKNOWN = 2
EOF = 99

# Token codes
INT_LIT = 10
IDENT = 11
ASSIGN_OP = 20
ADD_OP = 21
SUB_OP = 22
MULT_OP = 23
DIV_OP = 24
LEFT_PAREN = 25
RIGHT_PAREN = 26
SEMI_COLON = 27
ASSIGNMENT_OP = 28

#grammar correct
OK = 0
WARNING = 1
ERROR = 2

In [2]:
def getChar():
    # 다음 char를 nextChar에 저장하고 charClass 결정
    global nextChar, charClass, lastRow, lastColumn, index, lastIndex

    if index == lastIndex + 1:
        nextChar = "EOF"
    else:
        nextChar = chars[index]
    index = index + 1
    
    if nextChar == "EOF":
        charClass = EOF
    elif nextChar != ' ' and nextChar != "\n":
        if nextChar.isalpha():
            charClass = LETTER
        elif nextChar.isdigit():
            charClass = DIGIT
        else:
            charClass = UNKNOWN
    else:
        charClass = EOF

In [3]:
def addChar():
    # lexeme 배열에 nextChar를 덧붙임
    global lexeme, nextChar
    lexeme.append(nextChar)

In [4]:
def lexical():
    # 식별자, 숫자, 연산자를 구별해서 nextToken에 셋 중에 어떤 것인지 저장
    global lexeme, charClass, nextToken, idNum, constNum, opNum, lineNum, perLine, tokenString, idArray, constValue, grammar, message
    del lexeme[:]
    getNonBlank()
    
    # 처음 char가 문자로 시작하면 identifier를 의미함, lexeme 문자열을 tokenString에 저장
    if charClass == LETTER:
        addChar(); getChar()
        while charClass == LETTER or charClass == DIGIT:
            addChar(); getChar()
        nextToken = IDENT
        idNum = idNum + 1
        tokenString = ''.join(lexeme)
    
    # 처음 char가 숫자로 시작하면 const를 의미함, lexeme 문자열을 constValue에 저장
    elif charClass == DIGIT:
        addChar(); getChar()
        while charClass == DIGIT:
            addChar(); getChar()
        nextToken = INT_LIT
        constNum = constNum + 1
        constValue = int(''.join(lexeme))
    
    # 처음 char가 문자, 숫자가 아닌 special character로 시작하면 op를 의미함, lookup 함수 실행
    elif charClass == UNKNOWN:
        lookup(); getChar()
    
    # end of file
    elif charClass == EOF:
        nextToken = EOF
        lexeme.append('EOF0')
        print(perLine[lineNum])
        print('ID:', idNum, '; CONST:', constNum, '; OP:', opNum)
        if grammar == 0:
            print('(OK)')
        elif grammar == 1:
            print('(Warning)', message)
        elif grammar == 2:
            print('(Error)', message)

In [5]:
def lookup():
    # 문자와 숫자가 아닌 char를 받았을 때 operator(연산자) 구분
    global nextToken, nextChar, opNum, grammar, message
    if nextChar == '(':
        addChar(); nextToken = LEFT_PAREN
    elif nextChar == ')':
        addChar(); nextToken = RIGHT_PAREN
    elif nextChar == '+':
        addChar(); nextToken = ADD_OP
        opNum = opNum + 1
    elif nextChar == '-':
        addChar(); nextToken = SUB_OP
        opNum = opNum + 1
    elif nextChar == '*':
        addChar(); nextToken = MULT_OP
        opNum = opNum + 1
    elif nextChar == '/':
        addChar(); nextToken = DIV_OP
        opNum = opNum + 1
    elif nextChar == ';':
        addChar(); nextToken = SEMI_COLON
    elif nextChar == ':':
        addChar(); getChar()
        if nextChar == '=':
            addChar(); nextToken = ASSIGNMENT_OP
        else:
            # = 등호가 생략되었을 때 경고(Warning) 메세지 출력 후 오류 복구 
            lexeme.append('='); nextToken = ASSIGNMENT_OP
            grammar = 1; message = warningMessage(1)
    # : 기호가 생략되었을 경고(Warning) 메세지 출력 후 오류 복구 
    elif nextChar == '=':
        addChar(); nextToken = ASSIGNMENT_OP
        grammar = 1; message = warningMessage(6)
    else:
        addChar(); nextToken = EOF

In [6]:
def getNonBlank():
    # white-space 무시
    global nextChar
    while nextChar.isspace():
        getChar()

In [7]:
def program():
    # <program> → <statements>
    statements()
    
def statements():
    # <statements>→ <statement> { ; <statement> }
    global nextToken, idNum, constNum, opNum, lineNum, perLine, grammar, message
    idxNum = 0
    statement()
    # semi-colon(;) 있으면 해당 line의 결과 출력
    while nextToken == SEMI_COLON:
        print(perLine[lineNum])
        lineNum = lineNum + 1
        print('ID:', idNum, '; CONST:', constNum, '; OP:', opNum)
        if grammar == 0:
            print('(OK)')
        elif grammar == 1:
            print('(Warning)', message)
        elif grammar == 2:
            print('(Error)', message)
        grammar = 0
        message = ' '
        idNum = constNum = opNum = 0
        lexical()
        idxNum = idxNum + 1
        statement()
        
def statement():
    # <statement> → ident := <expression>
    global nextToken, tokenString, idArray, grammar, message
    if nextToken == IDENT:
        stmtKey = tokenString
        lexical()
        if nextToken == ASSIGNMENT_OP:
            lexical()
            stmtValue = expression()
            idArray[stmtKey] = stmtValue
        else:
            # 식별자 뒤에 등호가 없을 때 에러(Error) 메세지 출력 
            grammar = 2; message = errorMessage(2)
            lexical()
            idArray[stmtKey] = 'Unknown'
    else:
        # statement가 식별자로 시작하지 않을 때 에러(Error) 메세지 출력
        lexical()
        grammar = 2; message = errorMessage(3)
    
def expression():
    # <expression> → <term><term_tail>
    # term()과 term_tail()을 실행 후 식별자가 정의되었으면 + 또는 - 연산을 수행
    global add_op
    expValue = term()
    operand = term_tail()
    if expValue == 'Unknown':
        ;
    else:
        if add_op == ADD_OP:
            expValue = expValue + operand
        elif add_op == SUB_OP:
            expValue = expValue - operand
    add_op = 0
    return expValue
    
def term_tail():
    # <term_tail> → { (+ | -)<term> }
    # add 연산지인지 sub 연산자인지 기억하고 lexical()을 실행해 다시 nextToken을 받아와서 term() 함수 수행
    global nextToken, add_op
    tmtailValue = 0
    while nextToken == ADD_OP or nextToken == SUB_OP:
        if nextToken == ADD_OP:
            add_op = ADD_OP
        elif nextToken == SUB_OP:
            add_op = SUB_OP
        lexical()
        tmtailValue = term()
    return tmtailValue
        
        
def term():
    # <term> → <factor> <factor_tail>
    # factor()와 factor_tail()을 실행 후 식별자가 정의되었으면 * 또는 - 연산을 수행
    global mult_op
    tmValue = factor()
    operand = factor_tail()
    if tmValue == 'Unknown':
        ;
    else:
        if mult_op == MULT_OP:
            tmValue = tmValue * operand
        elif mult_op == DIV_OP:
            tmValue = tmValue / operand
    mult_op = 0
    return tmValue
    
def factor_tail():
    # <factor_tail> → { (* | /)<factor> }
    # mult 연산자인지 div 연산자인지 기억하고 lexical()을 실행해 다시 nextToken을 받아와서 factor() 함수 수행
    global nextToken, mult_op
    factailValue = 0
    while nextToken == MULT_OP or nextToken == DIV_OP:
        if nextToken == MULT_OP:
            mult_op = MULT_OP
        elif nextToken == DIV_OP:
            mult_op = DIV_OP
        lexical()
        factailValue = factor()
    return factailValue
        

def factor():
    # <factor> → ( <expression> ) | ident | const
    global nextToken, tokenString, constValue, expValue, grammar, message, opNum, nextChar, index
    # nextToken이 식별자일 때, 기존 symbol table에 있던 값이면 그에 대응되는 상수를 반환하고
    # 없던 값이면 경고(Warning) 메세지 출력 후 오류 복구 
    if nextToken == IDENT:
        lexical()
        if tokenString in idArray:
            return idArray[tokenString]
        else:
            grammar = 2; message = errorMessage(1)
            idArray[tokenString] = 'Unknown'
            return idArray[tokenString]
            
    # nextToken이 상수이면 상수를 반환
    elif nextToken == INT_LIT:
        lexical()
        return constValue
    
    # nextToken이 왼쪽 괄호이고 오른쪽 괄호로 닫히면 expression() 함수의 반환값을 반환
    # 오른쪽 괄호로 안닫혔으면 에러(Error) 메세지 출력 후 파싱 진행
    elif nextToken == LEFT_PAREN:
        lexical()
        facValue = expression()
        if nextToken == RIGHT_PAREN:
            lexical()
        elif nextToken == SEMI_COLON:
            index = index - 2
            nextToken = RIGHT_PAREN
            lexical()
            nextToken = SEMI_COLON
            grammar = 1; message = warningMessage(7)
        return facValue
    
    # 연산자가 두 개 이상 들어갔을 때 경고(Warning) 메세지 출력 후 오류 복구
    elif nextToken == ADD_OP:
        grammar = 1; message = warningMessage(2)
        opNum = opNum - 1
    elif nextToken == SUB_OP:
        grammar = 1; message = warningMessage(3)
        opNum = opNum - 1
    elif nextToken == MULT_OP:
        grammar = 1; message = warningMessage(4)
        opNum = opNum - 1
    elif nextToken == DIV_OP:
        grammar = 1; message = warningMessage(5)
        opNum = opNum - 1

In [8]:
def warningMessage(code):
    if code == 1:
        message = '생략된 등호(=) 삽입'
    elif code == 2:
        message = '중복 연산자(+) 제거'
    elif code == 3:
        message = '중복 연산자(-) 제거'
    elif code == 4:
        message = '중복 연산자(-) 제거'
    elif code == 5:
        message = '중복 연산자(-) 제거'
    elif code == 6:
        message = '생략된 기호(:) 삽입'
    elif code == 7:
        message = '오른쪽 괄호가 없음'
        
    return message

In [9]:
def errorMessage(code):
    if code == 1:
        message = '정의되지 않은 변수가 참조됨'
    elif code == 2:
        message = '식별자(identifier) 뒤에 등호(=)가 없음'
    elif code == 3:
        message = '문장이 식별자로 시작하지 않음'

    return message

# Main

In [10]:
try:
    with open(sys.argv[1], "r") as file:
        list_line = file.read()
        for line in range(0, len(list_line)):
            chars.append(list_line[line])
    
    lastIndex = len(list_line) - 1
    lineNum = len(list_line.split('\n'))
    
    for per in range(0, lineNum):
        perLine.append(list_line.split('\n')[per])
    lineNum = 0

    getChar()
    lexical()
    program()
    
    print('Result ==>', idArray)
        
except Exception as e:
    print('ERROR - ', e)
    sys.exit()

operand = 1;
ID: 1 ; CONST: 1 ; OP: 0
(Warning) 생략된 기호(:) 삽입
operand2 := (operand1 * 3 + 2 ;
ID: 2 ; CONST: 2 ; OP: 2
(Error) 생략된 오른쪽 괄호 삽입
target := operand1 + operand2 * 3 ;
ID: 3 ; CONST: 1 ; OP: 2
(OK)
goal := (target + operand2) / 7
ID: 3 ; CONST: 1 ; OP: 2
(OK)
Result ==> {'operand': 1, 'operand1': 'Unknown', 'operand2': 'Unknown', 'target': 'Unknown', 'goal': 'Unknown'}
